In [ ]:
import os
import threading
import datetime
import csv
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import serial
import serial.tools.list_ports
import queue
import time
import re
import webbrowser

# Column headers for Google Spreadsheet
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets Scope
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

def trigger_poke(serial_ports):
    """
    Sends the TRIGGER_POKE command to each connected FED3 device for identification.
    """
    for port in serial_ports:
        try:
            with serial.Serial(port, baudrate=115200, timeout=1) as ser:
                ser.write(b'TRIGGER_POKE\n')
                time.sleep(0.2)
                response = ser.readline().decode().strip()
                print(f"Response from {port}: {response}")
        except Exception as e:
            print(f"Error sending poke command to {port}: {e}")

class SplashScreen:
    def __init__(self, root, duration=5000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 0)
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        self.root.configure(bg="black")
        self.label_text = tk.Label(
            self.root,
            text="McCutcheonLab Technologies\nRTFED (Raspberry Pi OS)",
            font=("Cascadia Code", 40, "bold"),
            bg="black",
            fg="violet"
        )
        self.label_text.place(relx=0.5, rely=0.5, anchor=tk.CENTER)
        self.fade_in_out(duration)

    def fade_in_out(self, duration):
        self.fade_in(1000, lambda: self.fade_out(duration - 1000, self.close))

    def fade_in(self, time_ms, callback):
        alpha = 0.0
        increment = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha < 1.0:
                alpha += increment
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def fade_out(self, time_ms, callback):
        alpha = 1.0
        decrement = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha > 0.0:
                alpha -= decrement
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def close(self):
        self.root.destroy()

class FED3MonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("RTFED (Pi)")
        self.root.geometry("1200x800")
        self.dark_mode = False
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.data_queue = queue.Queue()
        self.serial_ports = set(self.detect_serial_ports())
        self.threads = []
        self.port_widgets = {}
        self.port_queues = {}
        self.port_threads = {}
        self.identification_threads = {}
        self.identification_stop_events = {}
        self.log_queue = queue.Queue()
        self.recording_circle = None
        self.recording_label = None
        self.data_to_save = {}
        self.stop_event = threading.Event()
        self.logging_active = False
        self.data_saved = False
        self.gspread_client = None
        self.last_device_check_time = time.time()
        self.retry_attempts = 5
        self.retry_delay = 2
        self.port_to_device_number = {}
        self.port_to_serial = {}
        self.time_sync_commands = {}

        # Mode selection variables
        self.mode_var = tk.StringVar(value="Select Mode")
        self.mode_options = [
            "0 - Free Feeding", "1 - FR1", "2 - FR3", "3 - FR5",
            "4 - Progressive Ratio", "5 - Extinction", "6 - Light Tracking",
            "7 - FR1 (Reversed)", "8 - PR (Reversed)", "9 - Self-Stim",
            "10 - Self-Stim (Reversed)", "11 - Timed Feeding", "12 - ClosedEconomy_PR1"
        ]

        self.setup_gui()
        self.root.after(0, self.update_gui)
        self.root.after(100, self.show_instruction_popup)
        self.root.after(200, self.start_identification_threads)
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def detect_serial_ports(self):
        ports = list(serial.tools.list_ports.comports())
        fed3_ports = []
        for port in ports:
            if port.vid == 0x239A and port.pid == 0x800B:
                fed3_ports.append(port.device)
        return fed3_ports

    def setup_gui(self):
        # Configure grid
        self.root.grid_columnconfigure((0,1,2,3), weight=1)
        self.root.grid_rowconfigure((0,1,2,3,4,5,6,7), weight=1)

        # Experimenter Name
        tk.Label(self.root, text="Your Name:", font=("Cascadia Code", 12, "bold")).grid(column=0, row=0, sticky=tk.E, padx=5, pady=5)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name, width=30)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=5, pady=5)

        # Experiment Name
        tk.Label(self.root, text="Experiment Name:", font=("Cascadia Code", 12, "bold")).grid(column=2, row=0, sticky=tk.E, padx=5, pady=5)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name, width=30)
        self.experiment_entry.grid(column=3, row=0, sticky=tk.W, padx=5, pady=5)

        # JSON File
        tk.Label(self.root, text="Google API JSON File:", font=("Cascadia Code", 12, "bold")).grid(column=0, row=1, sticky=tk.E, padx=5, pady=5)
        self.json_entry = ttk.Entry(self.root, textvariable=self.json_path, width=50)
        self.json_entry.grid(column=1, row=1, sticky=tk.W, padx=5, pady=5)
        self.browse_json_button = tk.Button(self.root, text="Browse JSON", font=("Cascadia Code", 10), command=self.browse_json)
        self.browse_json_button.grid(column=2, row=1, padx=5, pady=5, sticky=tk.W)

        # Spreadsheet ID and Start/Stop
        tk.Label(self.root, text="Google Spreadsheet ID:", font=("Cascadia Code", 12, "bold")).grid(column=0, row=2, sticky=tk.E, padx=5, pady=5)
        self.spreadsheet_entry = ttk.Entry(self.root, textvariable=self.spreadsheet_id, width=50)
        self.spreadsheet_entry.grid(column=1, row=2, sticky=tk.W, padx=5, pady=5)
        self.start_button = tk.Button(self.root, text="START", bg="green", fg="white", font=("Cascadia Code", 12, "bold"), command=self.start_logging)
        self.start_button.grid(column=2, row=2, padx=10, pady=5, sticky=tk.W)
        self.stop_button = tk.Button(self.root, text="STOP (SAVE & QUIT)", bg="red", fg="white", font=("Cascadia Code", 12, "bold"), command=self.stop_logging)
        self.stop_button.grid(column=3, row=2, padx=10, pady=5, sticky=tk.W)

        # Data folder browse, Identify, Sync, Dark Mode
        self.browse_button = tk.Button(self.root, text="Browse Data Folder", bg="gold", fg="blue", font=("Cascadia Code", 12, "bold"), command=self.browse_folder)
        self.browse_button.grid(column=0, row=3, padx=5, pady=5, sticky=tk.W)
        self.identify_devices_button = tk.Button(self.root, text="Identify Devices", bg="orange", fg="black", font=("Cascadia Code", 12, "bold"), command=self.identify_fed3_devices)
        self.identify_devices_button.grid(column=1, row=3, padx=5, pady=5, sticky=tk.W)
        self.sync_time_button = tk.Button(self.root, text="Sync FED3 Time", bg="blue", fg="white", font=("Cascadia Code", 12, "bold"), command=self.sync_all_device_times)
        self.sync_time_button.grid(column=2, row=3, padx=5, pady=5, sticky=tk.W)
        self.dark_mode_button = tk.Button(self.root, text="Toggle Dark Mode", font=("Cascadia Code", 10), command=self.toggle_dark_mode)
        self.dark_mode_button.grid(column=3, row=3, padx=5, pady=5, sticky=tk.W)

        # Mode selection
        self.mode_menu = ttk.Combobox(self.root, textvariable=self.mode_var, values=self.mode_options, width=30, state="readonly")
        self.mode_menu.grid(column=0, row=4, padx=5, pady=5, sticky=tk.W)
        self.set_mode_button = tk.Button(self.root, text="Set Mode", bg="darkorange", fg="black", font=("Cascadia Code", 12, "bold"), command=self.set_device_mode)
        self.set_mode_button.grid(column=1, row=4, padx=5, pady=5, sticky=tk.W)

        # Recording indicator
        indicator_frame = tk.Frame(self.root)
        indicator_frame.grid(column=2, row=4, columnspan=2, pady=5)
        self.canvas = tk.Canvas(indicator_frame, width=100, height=100)
        self.canvas.pack()
        self.recording_circle = self.canvas.create_oval(25, 25, 75, 75, fill="Orange")
        self.recording_label = self.canvas.create_text(50, 90, text="Standby", font=("Cascadia Code", 12), fill="black")

        # Ports frame
        ports_frame_container = tk.Frame(self.root)
        ports_frame_container.grid(column=0, row=5, columnspan=4, padx=10, pady=10, sticky=(tk.N, tk.S, tk.E, tk.W))
        ports_canvas = tk.Canvas(ports_frame_container)
        ports_scrollbar = ttk.Scrollbar(ports_frame_container, orient="vertical", command=ports_canvas.yview)
        ports_canvas.configure(yscrollcommand=ports_scrollbar.set)
        ports_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        ports_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.ports_frame = tk.Frame(ports_canvas)
        ports_canvas.create_window((0,0), window=self.ports_frame, anchor="nw")
        self.ports_frame.bind("<Configure>", lambda e: ports_canvas.configure(scrollregion=ports_canvas.bbox("all")))

        if not self.serial_ports:
            tk.Label(self.ports_frame, text="Connect your FED3 devices and restart the GUI!", font=("Cascadia Code", 14), fg="red").pack()
        else:
            for idx, port in enumerate(self.serial_ports):
                self.initialize_port_widgets(port, idx)

        # Log frame
        log_frame = tk.Frame(self.root)
        log_frame.grid(column=0, row=6, columnspan=4, padx=10, pady=10, sticky=(tk.N, tk.S, tk.E, tk.W))
        self.log_text = tk.Text(log_frame, height=10, width=150, font=("Cascadia Code", 10))
        self.log_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        log_scrollbar = ttk.Scrollbar(log_frame, orient="vertical", command=self.log_text.yview)
        log_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_text.configure(yscrollcommand=log_scrollbar.set)

        # Bottom frame
        bottom_frame = tk.Frame(self.root)
        bottom_frame.grid(column=0, row=7, columnspan=4, pady=5)
        tk.Label(bottom_frame, text="© 2025 McCutcheonLab | UiT | Norway", font=("Cascadia Code", 10), fg="royalblue").pack()
        hyperlink_label = tk.Label(bottom_frame, text="Developed by Hamid Taghipourbibalan", font=("Cascadia Code", 8, "italic"), fg="blue", cursor="hand2")
        hyperlink_label.pack()
        hyperlink_label.bind("<Button-1>", lambda e: webbrowser.open_new("https://www.linkedin.com/in/hamid-taghipourbibalan-b7239088/"))

        self.apply_theme()

    def browse_json(self):
        filename = filedialog.askopenfilename(title="Select JSON File", filetypes=[("JSON Files", "*.json")])
        if filename:
            self.json_path.set(filename)

    def browse_folder(self):
        folder = filedialog.askdirectory(title="Select Folder to Save Data")
        if folder:
            self.save_path = folder
            self.log_queue.put(f"Data folder selected: {folder}")

    def start_identification_threads(self):
        for port in list(self.serial_ports):
            self.start_identification_thread(port)

    def start_identification_thread(self, port):
        if port in self.identification_threads:
            return
        stop_event = threading.Event()
        self.identification_stop_events[port] = stop_event
        def delayed_ident():
            time.sleep(1)
            self.identification_thread(port, stop_event)
        t = threading.Thread(target=delayed_ident, daemon=True)
        t.start()
        self.identification_threads[port] = t
        self.log_queue.put(f"Started identification thread for {port}.")

    def stop_identification_threads(self):
        for port, evt in list(self.identification_stop_events.items()):
            evt.set()
        for port, t in list(self.identification_threads.items()):
            t.join()
            self.log_queue.put(f"Stopped identification thread for {port}.")
            del self.identification_threads[port]
            del self.identification_stop_events[port]

    def identification_thread(self, port, stop_event):
        event_index = column_headers.index("Event") - 1
        dn_index = column_headers.index("Device_Number") - 1
        device_num = None
        try:
            ser = serial.Serial(port, 115200, timeout=0.1)
            while not stop_event.is_set():
                line = ser.readline().decode('utf-8', errors='replace').strip()
                if line and "," in line:
                    parts = line.split(",")[1:]
                    if len(parts) == len(column_headers)-1:
                        evt = parts[event_index].strip()
                        dn = parts[dn_index].strip()
                        if evt in ["Right","Left","Pellet","LeftWithPellet","RightWithPellet"]:
                            self.port_queues[port].put("RIGHT_POKE")
                        if dn:
                            device_num = dn
                            break
        except Exception as e:
            self.log_queue.put(f"Error identifying {port}: {e}")
        finally:
            try: ser.close()
            except: pass
        if device_num:
            self.log_queue.put(f"Identified device_number={device_num} on port={port}")
            self.register_device_number(port, device_num)

    def identify_fed3_devices(self):
        self.stop_identification_threads()
        if not self.serial_ports:
            messagebox.showwarning("Warning","No FED3 devices detected. Connect devices first!")
            return
        self.log_queue.put("Identifying devices by triggering poke...")
        threading.Thread(target=lambda: trigger_poke(list(self.serial_ports)), daemon=True).start()
        # Restart identification after poke delay
        self.root.after(1000, self.start_identification_threads)

    def register_device_number(self, port, number):
        self.port_to_device_number[port] = number
        if self.logging_active:
            self.start_logging_for_port(port)

    def start_logging(self):
        self.stop_identification_threads()
        self.stop_event.clear()
        self.logging_active = True
        if not self.json_path.get() or not self.spreadsheet_id.get() or not self.save_path:
            messagebox.showerror("Error","Please provide JSON file, Spreadsheet ID, and data folder.")
            return
        # Sanitize names
        expfn = re.sub(r'[<>:"/\\|?*]','_',self.experimenter_name.get().strip().lower())
        expname = re.sub(r'[<>:"/\\|?*]','_',self.experiment_name.get().strip().lower())
        self.experimenter_name.set(expfn)
        self.experiment_name.set(expname)
        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            self.log_queue.put("Connected to Google Sheets!")
        except Exception as e:
            messagebox.showerror("Error",f"Failed to connect: {e}")
            return
        # Disable inputs
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        self.json_entry.config(state='disabled')
        self.spreadsheet_entry.config(state='disabled')
        self.browse_json_button.config(state='disabled')
        self.browse_button.config(state='disabled')
        self.start_button.config(state='disabled')
        # Update indicator
        self.canvas.itemconfig(self.recording_circle, fill="yellow")
        self.canvas.itemconfig(self.recording_label, text="Logging...", fill="black")
        # Start per-port logging threads
        for port in list(self.serial_ports):
            if port in self.port_to_device_number:
                self.start_logging_for_port(port)

    def start_logging_for_port(self, port):
        if port in self.port_threads:
            return
        if port not in self.port_to_device_number:
            self.log_queue.put(f"Cannot start logging for {port}, no device number.")
            return
        device_num = self.port_to_device_number[port]
        ws_name = f"Device_{device_num}"
        def attempt_conn():
            for i in range(self.retry_attempts):
                try:
                    ser = serial.Serial(port,115200,timeout=0.1)
                    self.port_to_serial[port] = ser
                    self.data_to_save[port] = []
                    t = threading.Thread(target=self.read_from_port, args=(ser,ws_name,port), daemon=True)
                    t.start()
                    self.port_threads[port] = t
                    self.port_widgets[port]['status_label'].config(text="Ready", fg="green")
                    self.log_queue.put(f"Started logging on {port} -> {ws_name}")
                    return
                except Exception as e:
                    self.log_queue.put(f"Attempt {i+1} error on {port}: {e}")
                    time.sleep(self.retry_delay)
            self.log_queue.put(f"Failed to connect to {port} after retries.")
        threading.Thread(target=attempt_conn, daemon=True).start()

    def read_from_port(self, ser, worksheet_name, port):
        # Open sheet
        sheet = None
        try:
            spreadsheet = self.gspread_client.open_by_key(self.spreadsheet_id.get())
            sheet = self.get_or_create_worksheet(spreadsheet, worksheet_name)
        except Exception as e:
            self.log_queue.put(f"Failed to open sheet for {port}: {e}")
        cached = []
        last_send = time.time()
        jam_flag = False
        event_idx = column_headers.index("Event")-1
        while not self.stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8', errors='replace').strip()
            except Exception as e:
                self.log_queue.put(f"Read error on {port}: {e}")
                break
            # Handle sync responses
            cmd = self.time_sync_commands.get(port)
            if cmd and cmd[0]=='pending':
                start_t = cmd[1]
                if line=="TIME_SET_OK":
                    self.log_queue.put(f"Time synced for {port}.")
                    self.time_sync_commands[port] = ('done',time.time())
                elif line=="TIME_SET_FAIL" or time.time()-start_t>2:
                    self.log_queue.put(f"Time sync no confirmation for {port}.")
                    self.time_sync_commands[port] = ('done',time.time())
                continue
            if line and "," in line:
                parts = line.split(",")[1:]
                if len(parts)==len(column_headers)-1:
                    evt = parts[event_idx].strip()
                    ts = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
                    row = [ts]+parts
                    cached.append(row)
                    self.data_to_save.setdefault(port,[]).append(row)
                    self.port_queues[port].put(f"Data logged: {parts}")
                    if evt=="JAM": jam_flag=True
                    if evt in ["Right","Pellet","LeftWithPellet","RightWithPellet"]:
                        self.port_queues[port].put("RIGHT_POKE")
            # Send batch to sheet
            if sheet and time.time()-last_send>5:
                if cached:
                    try:
                        sheet.append_rows(cached)
                        self.log_queue.put(f"Appended {len(cached)} rows from {port}.")
                        cached.clear()
                    except Exception as e:
                        self.log_queue.put(f"Append error for {port}: {e}")
                if jam_flag:
                    try:
                        jam_row = ['']*len(column_headers)
                        jam_row[0] = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
                        jam_row[column_headers.index("Event")] = "JAM"
                        jam_row[column_headers.index("Device_Number")] = self.port_to_device_number.get(port,'')
                        sheet.append_row(jam_row)
                        self.log_queue.put(f"JAM event logged for {port}")
                        jam_flag=False
                    except Exception as e:
                        self.log_queue.put(f"JAM append error for {port}: {e}")
                last_send = time.time()
            time.sleep(0.1)
        try: ser.close()
        except: pass
        self.log_queue.put(f"Closed serial port {port}")

    def get_or_create_worksheet(self, spreadsheet, title):
        try:
            return spreadsheet.worksheet(title)
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
            sheet.append_row(column_headers)
            return sheet

    def update_gui(self):
        # Port messages
        for port, q in self.port_queues.items():
            try:
                while True:
                    msg = q.get_nowait()
                    if msg=="RIGHT_POKE":
                        self.trigger_indicator(port)
                    else:
                        self.port_widgets[port]['text_widget'].insert(tk.END,msg+"\n")
                        self.port_widgets[port]['text_widget'].see(tk.END)
            except queue.Empty:
                pass
        # Log messages
        try:
            while True:
                log = self.log_queue.get_nowait()
                ts = datetime.datetime.now().strftime('%m/%d/%Y %H:%M:%S')
                self.log_text.insert(tk.END,f"{ts}: {log}\n")
                self.log_text.see(tk.END)
        except queue.Empty:
            pass
        # Check ports every 5s
        if time.time()-self.last_device_check_time>5:
            self.check_device_connections()
            self.last_device_check_time=time.time()
        self.root.after(100,self.update_gui)

    def check_device_connections(self):
        current = set(self.detect_serial_ports())
        # Disconnected
        for port in list(self.serial_ports):
            if port not in current:
                self.serial_ports.remove(port)
                self.port_widgets[port]['status_label'].config(text="Not Ready", fg="red")
                self.log_queue.put(f"Device on {port} disconnected.")
                self.identification_stop_events.get(port, threading.Event()).set()
                self.identification_threads.pop(port, None)
                self.port_threads.pop(port, None)
        # New
        for port in current:
            if port not in self.serial_ports:
                self.serial_ports.add(port)
                idx = len(self.port_widgets)
                self.initialize_port_widgets(port, idx)
                self.start_identification_thread(port)
                if self.logging_active and port in self.port_to_device_number:
                    self.start_logging_for_port(port)

    def trigger_indicator(self, port):
        canvas = self.port_widgets[port]['indicator_canvas']
        circle = self.port_widgets[port]['indicator_circle']
        def blink(n):
            if n>0:
                curr = canvas.itemcget(circle,'fill')
                nxt = 'red' if curr=='gray' else 'gray'
                canvas.itemconfig(circle, fill=nxt)
                self.root.after(250, lambda: blink(n-1))
            else:
                canvas.itemconfig(circle, fill='gray')
        blink(6)

    def set_device_mode(self):
        sel = self.mode_var.get()
        if not sel or sel=="Select Mode":
            messagebox.showerror("Error","Select a valid mode.")
            return
        mode_num = int(sel.split(" - ")[0])
        ports = [p for p,w in self.port_widgets.items() if w.get('selected_var',tk.BooleanVar()).get()]
        if not ports:
            messagebox.showwarning("No Ports","Select at least one port.")
            return
        for port in ports:
            try:
                with serial.Serial(port, baudrate=115200, timeout=2) as ser:
                    ser.write(f"SET_MODE:{mode_num}\n".encode())
                    start = time.time()
                    ok=False
                    while time.time()-start<3:
                        resp = ser.readline().decode().strip()
                        if resp=="MODE_SET_OK":
                            self.log_queue.put(f"Mode {mode_num} set on {port}.")
                            ok=True
                            break
                        elif resp=="MODE_SET_FAIL":
                            self.log_queue.put(f"Mode set failed on {port}.")
                            ok=True
                            break
                    if not ok:
                        self.log_queue.put(f"No confirmation from {port}.")
            except Exception as e:
                self.log_queue.put(f"Error setting mode on {port}: {e}")

    def sync_all_device_times(self):
        now = datetime.datetime.now()
        timestr = f"SET_TIME:{now.year},{now.month},{now.day},{now.hour},{now.minute},{now.second}"
        if self.logging_active:
            for port, ser in self.port_to_serial.items():
                if ser and ser.is_open:
                    self.time_sync_commands[port] = ('pending', time.time())
                    try:
                        ser.write((timestr+"\n").encode())
                    except Exception as e:
                        self.log_queue.put(f"Failed to sync {port}: {e}")
        else:
            for port in self.port_to_device_number:
                self.time_sync_commands[port] = ('pending', time.time())
                try:
                    with serial.Serial(port,115200,timeout=2) as ser:
                        ser.write((timestr+"\n").encode())
                        start = time.time()
                        got=False
                        while time.time()-start<2:
                            l = ser.readline().decode().strip()
                            if l=="TIME_SET_OK":
                                self.log_queue.put(f"Time synced for {port}.")
                                got=True
                                break
                            elif l=="TIME_SET_FAIL":
                                self.log_queue.put(f"Sync no confirm for {port}.")
                                got=True
                                break
                        if not got:
                            self.log_queue.put(f"Sync no confirm for {port}.")
                except Exception as e:
                    self.log_queue.put(f"Failed to sync {port}: {e}")
                finally:
                    self.time_sync_commands[port] = ('done',time.time())

    def toggle_dark_mode(self):
        self.dark_mode = not self.dark_mode
        self.apply_theme()

    def apply_theme(self):
        if self.dark_mode:
            bg='#2e2e2e'; fg='#ffffff'; ebg='#3c3c3c'; efg='#ffffff'; tbg='#3c3c3c'; tfg='#ffffff'; cbg='#2e2e2e'
        else:
            bg='white'; fg='black'; ebg='white'; efg='black'; tbg='white'; tfg='black'; cbg='white'
        self.root.configure(bg=bg)
        for widget in self.root.winfo_children():
            try: widget.configure(bg=bg, fg=fg)
            except: pass
        # Text widgets
        try:
            self.log_text.configure(bg=tbg, fg=tfg, insertbackground=fg)
        except: pass
        # Canvas
        try:
            self.canvas.configure(bg=cbg)
        except: pass

    def on_closing(self):
        if not self.data_saved:
            self.stop_identification_threads()
            self.stop_event.set()
            self.logging_active = False
            for t in list(self.identification_threads.values()): t.join()
            for t in list(self.port_threads.values()): t.join()
            self.save_all_data()
        self.root.destroy()

    def save_all_data(self):
        if not self.save_path:
            self.log_queue.put("No save path.")
            return
        ts = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        expfn = self.experimenter_name.get(); expnm = self.experiment_name.get()
        base = os.path.join(self.save_path, expfn, f"{expnm}_{ts}")
        os.makedirs(base, exist_ok=True)
        for port, rows in self.data_to_save.items():
            if not rows: continue
            pn = os.path.basename(port)
            safe = re.sub(r'[<>:"/\\|?*]','_',pn)
            dn = self.port_to_device_number.get(port,'unknown')
            fn = os.path.join(base, f"{safe}_device_{dn}_{ts}.csv")
            try:
                with open(fn,'w',newline='') as f:
                    w=csv.writer(f)
                    w.writerow(column_headers)
                    w.writerows(rows)
                self.log_queue.put(f"Saved data for {port} to {fn}")
            except Exception as e:
                self.log_queue.put(f"Failed saving {port}: {e}")

if __name__ == "__main__":
    splash = tk.Tk()
    SplashScreen(splash, duration=5000)
    splash.after(5000, splash.destroy)
    splash.mainloop()
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


In [ ]:
import os
import threading
import datetime
import csv
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import serial
import serial.tools.list_ports
import queue
import time
import re
import webbrowser

# Column headers for Google Spreadsheet
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets Scope
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

def trigger_poke(serial_ports):
    # Sends the TRIGGER_POKE command to each port.
    for port in serial_ports:
        try:
            with serial.Serial(port, baudrate=115200, timeout=1) as ser:
                ser.write(b'TRIGGER_POKE\n')
                time.sleep(0.2)
                response = ser.readline().decode().strip()
                print(f"Response from {port}: {response}")
        except Exception as e:
            print(f"Error sending poke command to {port}: {e}")

class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 1)
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        self.root.configure(bg="black")
        self.label = tk.Label(
            self.root,
            text="McCutcheonLab Technologies\nRTFED (Raspberry Pi OS)",
            font=("Cascadia Code", 32, "bold"),
            bg="black",
            fg="violet"
        )
        self.label.pack(expand=True)
        self.fade_in_out(duration)

    def fade_in_out(self, duration):
        self.fade_in(1000, lambda: self.fade_out(2000, self.close))

    def fade_in(self, time_ms, callback):
        alpha = 0.0
        increment = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha < 1.0:
                alpha += increment
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def fade_out(self, time_ms, callback):
        alpha = 1.0
        decrement = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha > 0.0:
                alpha -= decrement
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def close(self):
        self.root.destroy()

class FED3MonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("RTFED (Pi)")
        self.root.geometry("1200x800")
        # State vars
        self.dark_mode = False
        self.experimenter_name = tk.StringVar()
        self.experiment_name   = tk.StringVar()
        self.json_path         = tk.StringVar()
        self.spreadsheet_id    = tk.StringVar()
        self.save_path         = ""
        self.data_queue        = queue.Queue()
        self.serial_ports      = set(self.detect_serial_ports())
        self.port_widgets      = {}
        self.port_queues       = {}
        self.port_threads      = {}
        self.identification_threads   = {}
        self.identification_stop_events= {}
        self.log_queue         = queue.Queue()
        self.recording_circle  = None
        self.recording_label   = None
        self.data_to_save      = {}
        self.stop_event        = threading.Event()
        self.logging_active    = False
        self.data_saved        = False
        self.gspread_client    = None
        self.last_device_check_time = time.time()
        self.retry_attempts    = 5
        self.retry_delay       = 2
        self.port_to_device_number    = {}
        self.port_to_serial    = {}
        self.time_sync_commands = {}
        # Mode selection
        self.mode_var          = tk.StringVar(value="Select Mode")
        self.mode_options      = [
            "0 - Free Feeding", "1 - FR1", "2 - FR3", "3 - FR5",
            "4 - Progressive Ratio", "5 - Extinction", "6 - Light Tracking",
            "7 - FR1 (Reversed)", "8 - PR (Reversed)", "9 - Self-Stim",
            "10 - Self-Stim (Reversed)", "11 - Timed Feeding", "12 - ClosedEconomy_PR1"
        ]

        # Build UI
        self.setup_gui()
        self.root.after(0,   self.update_gui)
        self.root.after(100, self.show_instruction_popup)
        self.root.after(200, self.start_identification_threads)
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def detect_serial_ports(self):
        ports = list(serial.tools.list_ports.comports())
        fed3_ports = []
        for port in ports:
            if port.vid == 0x239A and port.pid == 0x800B:
                fed3_ports.append(port.device)
        return fed3_ports

    def setup_gui(self):
        # Grid layout
        self.root.grid_columnconfigure((0,1,2,3), weight=1)
        self.root.grid_rowconfigure((0,1,2,3,4,5,6), weight=1)
        # Experimenter & Experiment
        tk.Label(self.root, text="Your Name:", font=("Cascadia Code",12,"bold")).grid(column=0,row=0,sticky=tk.E,padx=5,pady=5)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name, width=30)
        self.experimenter_entry.grid(column=1,row=0,sticky=tk.W,padx=5,pady=5)
        tk.Label(self.root, text="Experiment Name:", font=("Cascadia Code",12,"bold")).grid(column=2,row=0,sticky=tk.E,padx=5,pady=5)
        self.experiment_entry   = ttk.Entry(self.root, textvariable=self.experiment_name, width=30)
        self.experiment_entry.grid(column=3,row=0,sticky=tk.W,padx=5,pady=5)
        # JSON file
        tk.Label(self.root, text="Google API JSON File:", font=("Cascadia Code",12,"bold")).grid(column=0,row=1,sticky=tk.E,padx=5,pady=5)
        self.json_entry = ttk.Entry(self.root, textvariable=self.json_path, width=50)
        self.json_entry.grid(column=1,row=1,sticky=tk.W,padx=5,pady=5)
        self.browse_json_button = tk.Button(self.root, text="Browse", command=self.browse_json, font=("Cascadia Code",10))
        self.browse_json_button.grid(column=2,row=1,sticky=tk.W,padx=5,pady=5)
        # Spreadsheet ID
        tk.Label(self.root, text="Spreadsheet ID:", font=("Cascadia Code",12,"bold")).grid(column=0,row=2,sticky=tk.E,padx=5,pady=5)
        self.spreadsheet_entry = ttk.Entry(self.root, textvariable=self.spreadsheet_id, width=50)
        self.spreadsheet_entry.grid(column=1,row=2,sticky=tk.W,padx=5,pady=5)
        # Data folder
        self.browse_button = tk.Button(self.root, text="Select Data Folder", command=self.browse_folder, font=("Cascadia Code",12,"bold"), bg="gold",fg="blue")
        self.browse_button.grid(column=0,row=3,sticky=tk.W,padx=10,pady=5)
        # Start / Stop
        self.start_button = tk.Button(self.root, text="START", command=self.start_logging, font=("Cascadia Code",12,"bold"), bg="green", fg="white")
        self.start_button.grid(column=2,row=2,sticky=tk.W,padx=10,pady=5)
        self.stop_button  = tk.Button(self.root, text="STOP & SAVE", command=self.stop_logging, font=("Cascadia Code",12,"bold"), bg="red", fg="white")
        self.stop_button.grid(column=3,row=2,sticky=tk.W,padx=10,pady=5)
        # Sync Time & Identify & Mode & Dark
        self.sync_time_button = tk.Button(self.root, text="Sync FED3 Time", command=self.sync_all_device_times, font=("Cascadia Code",10,"bold"), bg="blue", fg="white")
        self.sync_time_button.grid(column=0,row=4,padx=5,pady=5,sticky=tk.W)
        self.identify_button  = tk.Button(self.root, text="Identify Devices", command=self.identify_fed3_devices, font=("Cascadia Code",10,"bold"), bg="orange", fg="black")
        self.identify_button.grid(column=1,row=4,padx=5,pady=5,sticky=tk.W)
        self.dark_mode_button = tk.Button(self.root, text="Toggle Dark Mode", command=self.toggle_dark_mode, font=("Cascadia Code",10))
        self.dark_mode_button.grid(column=3,row=4,padx=5,pady=5,sticky=tk.E)
        # Mode selection combobox and button
        self.mode_menu = ttk.Combobox(self.root, textvariable=self.mode_var, values=self.mode_options, state="readonly", width=30)
        self.mode_menu.grid(column=1,row=5,padx=5,pady=5,sticky=tk.W)
        self.set_mode_button = tk.Button(self.root, text="Set Mode", command=self.set_device_mode, font=("Cascadia Code",10,"bold"), bg="darkorange", fg="black")
        self.set_mode_button.grid(column=2,row=5,padx=5,pady=5,sticky=tk.W)
        # Recording indicator
        indicator_frame = tk.Frame(self.root)
        indicator_frame.grid(column=1,row=3,columnspan=2,pady=10)
        self.canvas = tk.Canvas(indicator_frame,width=100,height=100)
        self.canvas.pack()
        self.recording_circle = self.canvas.create_oval(25,25,75,75,fill="Orange")
        self.recording_label  = self.canvas.create_text(50,90,text="Standby",font=("Cascadia Code",12),fill="black")
        # Ports list
        ports_container = tk.Frame(self.root)
        ports_container.grid(column=0,row=6,columnspan=4,pady=20,sticky=(tk.N,tk.S,tk.E,tk.W))
        ports_canvas    = tk.Canvas(ports_container)
        scrollbar       = ttk.Scrollbar(ports_container,orient="vertical",command=ports_canvas.yview)
        ports_canvas.configure(yscrollcommand=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT,fill=tk.Y)
        ports_canvas.pack(side=tk.LEFT,fill=tk.BOTH,expand=True)
        self.ports_frame = tk.Frame(ports_canvas)
        ports_canvas.create_window((0,0),window=self.ports_frame,anchor="nw")
        self.ports_frame.bind("<Configure>",lambda e: ports_canvas.configure(scrollregion=ports_canvas.bbox("all")))
        if not self.serial_ports:
            tk.Label(self.ports_frame,text="Connect your FED3 devices and restart!",font=("Cascadia Code",14),fg="red").pack()
        else:
            for idx,port in enumerate(self.serial_ports):
                self.initialize_port_widgets(port, idx)
        # Log pane
        log_frame = tk.Frame(self.root)
        log_frame.grid(column=0,row=7,columnspan=4,sticky=(tk.N,tk.S,tk.E,tk.W), pady=5)
        self.log_text = tk.Text(log_frame,height=10,width=130,font=("Cascadia Code",10))
        self.log_text.pack(side=tk.LEFT,fill=tk.BOTH,expand=True)
        log_scrollbar = ttk.Scrollbar(log_frame, orient="vertical", command=self.log_text.yview)
        log_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_text.configure(yscrollcommand=log_scrollbar.set)
        # Footer
        bottom = tk.Frame(self.root)
        bottom.grid(column=0,row=8,columnspan=4,pady=5,sticky=(tk.S,tk.E,tk.W))
        tk.Label(bottom, text="© 2025 McCutcheonLab | UiT | Norway", font=("Cascadia Code",10), fg="royalblue").pack()
        link = tk.Label(bottom, text="Developed by Hamid Taghipourbibalan", font=("Cascadia Code",8,"italic"), fg="blue", cursor="hand2")
        link.pack()
        link.bind("<Button-1>", lambda e: self.open_hyperlink("https://www.linkedin.com/in/hamid-taghipourbibalan-b7239088/"))
        # Initial theming
        self.apply_theme()

    def open_hyperlink(self, URL):
        webbrowser.open_new(URL)

    def show_instruction_popup(self):
        messagebox.showinfo(
            "Instructions", 
            "1) Have you flashed your FED3 with the RTFED library? You should see the RTS label on your FED3 screen.\n"
            "2) Press the [Identify Devices] button to trigger a poke command on each FED3 and identify devices. If set on Free Feeding mode, wait for the Pellet event to Timeout (60 sec elapsed) before identifying the devices.\n"
            "3) After devices are identified, a) You can press [Sync FED3 Time] to synchronize time on all your FED3 devices! b) You can click on the check-box of the ports you'd like to change the modes, select the desired mode from the drop-down menu and change it."
        )
        messagebox.showwarning(
            "Caution", 
            "1) If you need to restart a FED3 during an experiment, do it while the internet connection is active.\n"
            "2) Restart and reconnect FED3 units one at a time if needed.\n"
            "3) After restarting a device, the data file saved locally would only contain the data logged after restart; however the full length data remains in your Google spreadsheet.\n"
            "4) IT IS VERY IMPORTANT to identify FED3 devices before pressing START or else RTFED will not log data.\n"
            "5) We recommend using a powered USB hub if many FED3 units are connected."
        )

    def initialize_port_widgets(self, port, idx):
        if port in self.port_widgets:
            return
        frame = tk.LabelFrame(self.ports_frame, text=f"Port {os.path.basename(port)}", font=("Cascadia Code",10,"bold"))
        frame.grid(column=idx%2, row=idx//2, padx=10, pady=10, sticky=tk.W)
        status = tk.Label(frame, text="Not Ready", font=("Cascadia Code",10,"italic"), fg="red")
        status.grid(column=0,row=0,sticky=tk.W)
        textw  = tk.Text(frame,width=40,height=6,font=("Cascadia Code",9))
        textw.grid(column=0,row=1,sticky=(tk.N,tk.S,tk.E,tk.W))
        ind_can= tk.Canvas(frame,width=20,height=20)
        ind_can.grid(column=1,row=0,padx=5)
        ind_cir= ind_can.create_oval(5,5,15,15,fill="gray")
        # Checkbox for mode apply
        sel_var = tk.BooleanVar(value=True)
        chk = tk.Checkbutton(frame, text="Apply Mode", variable=sel_var, font=("Cascadia Code",10))
        chk.grid(column=0,row=2,sticky=tk.W,pady=(5,0))
        self.port_widgets[port] = {
            'frame': frame, 'status_label': status,
            'text_widget': textw,
            'indicator_canvas': ind_can, 'indicator_circle': ind_cir,
            'selected_var': sel_var
        }
        self.port_queues[port] = queue.Queue()
        # test port
        try:
            ser = serial.Serial(port,115200,timeout=1); ser.close()
            status.config(text="Ready", fg="green")
        except Exception as e:
            status.config(text="Not Ready", fg="red")
            self.log_queue.put(f"Error with port {port}: {e}")

    def browse_json(self):
        fname = filedialog.askopenfilename(title="Select JSON file", filetypes=[("JSON","*.json")])
        if fname:
            self.json_path.set(fname)

    def browse_folder(self):
        folder = filedialog.askdirectory(title="Select folder to save data")
        if folder:
            self.save_path = folder
            self.log_queue.put(f"Data folder: {folder}")

    def start_identification_threads(self):
        for port in list(self.serial_ports):
            self.start_identification_thread(port)

    def start_identification_thread(self, port):
        if port in self.identification_threads:
            return
        stop_evt = threading.Event()
        self.identification_stop_events[port] = stop_evt
        def run_ident():
            time.sleep(1)
            self.identification_thread(port, stop_evt)
        t = threading.Thread(target=run_ident); t.daemon=True; t.start()
        self.identification_threads[port] = t
        self.log_queue.put(f"Started identification for {port}")

    def stop_identification_threads(self):
        for port,evt in self.identification_stop_events.items():
            evt.set()
        for port,t in self.identification_threads.items():
            t.join()
        self.identification_threads.clear(); self.identification_stop_events.clear()

    def identification_thread(self, port, stop_event):
        event_idx = column_headers.index("Event")-1
        dn_idx    = column_headers.index("Device_Number")-1
        device_num=None
        try:
            ser = serial.Serial(port,115200,timeout=0.1)
            while not stop_event.is_set():
                line = ser.readline().decode('utf-8',errors='replace').strip()
                if line and "," in line:
                    parts = line.split(",")[1:]
                    if len(parts)==len(column_headers)-1:
                        ev = parts[event_idx].strip()
                        dn = parts[dn_idx].strip()
                        if ev in ["Right","Left","Pellet","LeftWithPellet","RightWithPellet"]:
                            self.port_queues[port].put("RIGHT_POKE")
                        if dn:
                            device_num=dn
                            break
        except Exception:
            pass
        finally:
            try: ser.close()
            except: pass
        if device_num:
            self.log_queue.put(f"Identified {device_num} on {port}")
            self.register_device_number(port,device_num)

    def register_device_number(self, port, device_number):
        self.port_to_device_number[port] = device_number
        if self.logging_active:
            self.start_logging_for_port(port)

    def identify_fed3_devices(self):
        self.stop_identification_threads()
        if not self.serial_ports:
            messagebox.showwarning("Warning","No FED3 devices detected.")
            return
        threading.Thread(target=self.trigger_poke_for_identification, daemon=True).start()

    def trigger_poke_for_identification(self):
        self.log_queue.put("Triggering poke for ID...")
        for port in list(self.serial_ports):
            try:
                with serial.Serial(port,115200,timeout=1) as ser:
                    ser.write(b'TRIGGER_POKE\n')
                    start=time.time(); dn=None
                    while time.time()-start<3:
                        line=ser.readline().decode('utf-8',errors='replace').strip()
                        if line and "," in line:
                            parts=line.split(",")
                            if len(parts)>=6:
                                dn=parts[5].strip()
                                break
                    if dn:
                        self.register_device_number(port,dn)
                        self.log_queue.put(f"Identified {dn} on {port}")
                    else:
                        self.log_queue.put(f"No device number from {port}")
            except Exception as e:
                self.log_queue.put(f"Poke error on {port}: {e}")
        self.log_queue.put("ID complete.")

    def start_logging(self):
        self.stop_identification_threads()
        self.stop_event.clear()
        self.logging_active = True
        # validate
        if not self.json_path.get() or not self.spreadsheet_id.get() or not self.save_path:
            messagebox.showerror("Error","Provide JSON file, Spreadsheet ID, and data folder.")
            return
        # sanitize names
        en = re.sub(r'[<>:"/\\|?*]','_', self.experimenter_name.get().strip().lower())
        ex = re.sub(r'[<>:"/\\|?*]','_', self.experiment_name.get().strip().lower())
        self.experimenter_name.set(en); self.experiment_name.set(ex)
        # connect sheets
        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            self.log_queue.put("Connected to Google Sheets!")
        except Exception as e:
            messagebox.showerror("Error",f"Google Sheets connection failed: {e}")
            return
        self.disable_input_fields()
        self.canvas.itemconfig(self.recording_circle, fill="yellow")
        self.canvas.itemconfig(self.recording_label, text="Logging...", fill="black")
        for port in list(self.serial_ports):
            if port in self.port_to_device_number:
                self.start_logging_for_port(port)

    def disable_input_fields(self):
        for w in [self.experimenter_entry, self.experiment_entry, self.json_entry, self.spreadsheet_entry, self.browse_json_button, self.browse_button, self.start_button]:
            w.config(state='disabled')

    def enable_input_fields(self):
        for w in [self.experimenter_entry, self.experiment_entry, self.json_entry, self.spreadsheet_entry, self.browse_json_button, self.browse_button, self.start_button]:
            w.config(state='normal')

    def start_logging_for_port(self, port):
        if port in self.port_threads:
            return
        if port not in self.port_to_device_number:
            self.log_queue.put(f"No device_number for {port}")
            return
        dn = self.port_to_device_number[port]
        ws = f"Device_{dn}"
        def try_conn():
            for i in range(self.retry_attempts):
                try:
                    ser = serial.Serial(port,115200,timeout=0.1)
                    self.port_to_serial[port]=ser
                    self.data_to_save[port]=[]
                    t = threading.Thread(target=self.read_from_port, args=(ser,ws,port))
                    t.daemon=True; t.start()
                    self.port_threads[port]=t
                    self.port_widgets[port]['status_label'].config(text="Ready", fg="green")
                    self.log_queue.put(f"Logging from {port} → {ws}")
                    return
                except Exception as e:
                    self.log_queue.put(f"Attempt {i+1} failed on {port}: {e}")
                    time.sleep(self.retry_delay)
            self.log_queue.put(f"Could not open {port}")
        threading.Thread(target=try_conn, daemon=True).start()

    def stop_logging(self):
        self.stop_event.set()
        self.logging_active = False
        self.log_queue.put("Stopping logging…")
        self.canvas.itemconfig(self.recording_circle, fill="red")
        self.canvas.itemconfig(self.recording_label, text="OFF", fill="red")
        self.enable_input_fields()
        threading.Thread(target=self._join_and_save, daemon=True).start()

    def _join_and_save(self):
        for t in list(self.port_threads.values()):
            t.join()
        self.save_all_data()
        self.data_saved=True
        self.root.after(0, self._final_exit)

    def _final_exit(self):
        messagebox.showinfo("Done","Data saved locally.")
        self.root.destroy()

    def save_all_data(self):
        if not self.save_path:
            self.log_queue.put("No save path!")
            return
        now = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        en = self.experimenter_name.get(); ex= self.experiment_name.get()
        base = os.path.join(self.save_path,en,f"{ex}_{now}")
        os.makedirs(base, exist_ok=True)
        for port,rows in self.data_to_save.items():
            dn = self.port_to_device_number.get(port,"unknown")
            safe = re.sub(r'[<>:"/\\|?*]','_',os.path.basename(port))
            fn = os.path.join(base,f"{safe}_device_{dn}_{now}.csv")
            try:
                with open(fn,'w',newline='') as f:
                    w=csv.writer(f); w.writerow(column_headers); w.writerows(rows)
                self.log_queue.put(f"Saved {port} → {fn}")
            except Exception as e:
                self.log_queue.put(f"Save error {port}: {e}")

    def update_gui(self):
        # port messages
        for port,q in list(self.port_queues.items()):
            try:
                while True:
                    m=q.get_nowait()
                    if m=="RIGHT_POKE":
                        self.trigger_indicator(port)
                    else:
                        w=self.port_widgets[port]['text_widget']
                        w.insert(tk.END,m+"\n"); w.see(tk.END)
            except queue.Empty:
                pass
        # log messages
        try:
            while True:
                lm=self.log_queue.get_nowait()
                ts=datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S")
                self.log_text.insert(tk.END,f"{ts}: {lm}\n"); self.log_text.see(tk.END)
        except queue.Empty:
            pass
        # device connect check
        if time.time()-self.last_device_check_time>5:
            self.check_device_connections()
            self.last_device_check_time=time.time()
        self.root.after(100, self.update_gui)

    def check_device_connections(self):
        current = set(self.detect_serial_ports())
        # removed
        for port in list(self.serial_ports - current):
            self.serial_ports.remove(port)
            self.port_widgets[port]['status_label'].config(text="Not Ready", fg="red")
            self.log_queue.put(f"Disconnected {port}")
            self.port_threads.pop(port,None)
            evt=self.identification_stop_events.pop(port,None)
            if evt: evt.set()
        # added
        for port in current - self.serial_ports:
            self.serial_ports.add(port)
            idx=len(self.port_widgets)
            self.initialize_port_widgets(port,idx)
            self.start_identification_thread(port)
            if self.logging_active and port in self.port_to_device_number:
                self.start_logging_for_port(port)

    def read_from_port(self, ser, worksheet_name, port):
        sheet=None; cache=[]; send_int=5; last=time.time(); jam=False
        ev_idx = column_headers.index("Event")-1
        dn_idx = column_headers.index("Device_Number")-1
        dn = self.port_to_device_number.get(port,"unknown")
        try:
            ss = self.gspread_client.open_by_key(self.spreadsheet_id.get())
            sheet = self.get_or_create_worksheet(ss, worksheet_name)
        except Exception as e:
            self.log_queue.put(f"Sheet error {port}: {e}")
        while not self.stop_event.is_set():
            try:
                data=ser.readline().decode('utf-8',errors='replace').strip()
            except Exception as e:
                self.log_queue.put(f"Disconnect {port}: {e}")
                break
            if data and "," in data:
                parts=data.split(",")[1:]
                if len(parts)==len(column_headers)-1:
                    ev=parts[ev_idx].strip()
                    ts=datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
                    row=[ts]+parts
                    cache.append(row)
                    self.data_to_save.setdefault(port,[]).append(row)
                    if port in self.port_queues and ev in ["Right","Pellet","Left","LeftWithPellet","RightWithPellet"]:
                        self.port_queues[port].put("RIGHT_POKE")
                    if ev=="JAM":
                        jam=True
                else:
                    self.log_queue.put(f"Length mismatch {port}")
            now=time.time()
            if now-last>=send_int and sheet:
                if cache:
                    try:
                        sheet.append_rows(cache)
                        self.log_queue.put(f"Sent {len(cache)} rows from {port}")
                        cache.clear()
                    except Exception as e:
                        self.log_queue.put(f"Send error {port}: {e}")
                if jam:
                    try:
                        jr=['']*len(column_headers)
                        jr[0]=datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
                        jr[column_headers.index("Event")] = "JAM"
                        jr[column_headers.index("Device_Number")] = dn
                        sheet.append_row(jr)
                        self.log_queue.put(f"JAM logged {port}")
                    except Exception as e:
                        self.log_queue.put(f"JAM send error {port}: {e}")
                    jam=False
                last=now
            time.sleep(0.1)
        try: ser.close()
        except: pass
        self.log_queue.put(f"Closed {port}")

    def get_or_create_worksheet(self, spreadsheet, title):
        try:
            return spreadsheet.worksheet(title)
        except gspread.exceptions.WorksheetNotFound:
            sh = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
            sh.append_row(column_headers)
            return sh

    def trigger_indicator(self, port):
        w = self.port_widgets[port]['indicator_canvas']
        c = self.port_widgets[port]['indicator_circle']
        def blink(n):
            if n>0:
                current = w.itemcget(c,'fill')
                nxt = 'red' if current=='gray' else 'gray'
                w.itemconfig(c,fill=nxt)
                self.root.after(250, lambda: blink(n-1))
            else:
                w.itemconfig(c,fill='gray')
        blink(6)

    def set_device_mode(self):
        mode = self.mode_var.get()
        if not mode or mode.startswith("Select"):
            messagebox.showerror("Error","Select a mode first.")
            return
        mode_num = int(mode.split(" - ")[0])
        ports = [p for p,w in self.port_widgets.items() if w['selected_var'].get()]
        if not ports:
            messagebox.showwarning("Warning","Check at least one port.")
            return
        for port in ports:
            try:
                with serial.Serial(port,115200,timeout=2) as ser:
                    ser.write(f"SET_MODE:{mode_num}\n".encode())
                    start=time.time(); ok=False
                    while time.time()-start<3:
                        resp=ser.readline().decode().strip()
                        if resp=="MODE_SET_OK":
                            self.log_queue.put(f"Mode {mode_num} set on {port}")
                            ok=True; break
                        if resp=="MODE_SET_FAIL":
                            self.log_queue.put(f"Mode fail on {port}")
                            ok=True; break
                    if not ok:
                        self.log_queue.put(f"No response setting mode on {port}")
            except Exception as e:
                self.log_queue.put(f"Mode error on {port}: {e}")

    def sync_all_device_times(self):
        now=datetime.datetime.now()
        cmd=f"SET_TIME:{now.year},{now.month},{now.day},{now.hour},{now.minute},{now.second}"
        for port in self.port_to_device_number:
            try:
                if port in self.port_to_serial and self.port_to_serial[port].is_open:
                    ser=self.port_to_serial[port]
                else:
                    ser=serial.Serial(port,115200,timeout=2)
                ser.write((cmd+"\n").encode())
                start=time.time(); got=False
                while time.time()-start<2:
                    line=ser.readline().decode().strip()
                    if line=="TIME_SET_OK":
                        self.log_queue.put(f"Time synced {port}")
                        got=True; break
                    if line=="TIME_SET_FAIL":
                        self.log_queue.put(f"No confirm sync {port}")
                        got=True; break
                if not got:
                    self.log_queue.put(f"No response sync {port}")
                if ser not in self.port_to_serial.values():
                    ser.close()
            except Exception as e:
                self.log_queue.put(f"Sync error {port}: {e}")

    def toggle_dark_mode(self):
        self.dark_mode = not self.dark_mode
        self.apply_theme()

    def apply_theme(self):
        if self.dark_mode:
            bg="#2e2e2e"; fg="#ffffff"; ebg="#3c3c3c"; efg="#ffffff"; tbg="#3c3c3c"; tfg="#ffffff"; cbg="#2e2e2e"
        else:
            bg="white"; fg="black"; ebg="white"; efg="black"; tbg="white"; tfg="black"; cbg="white"
        # root & frames
        for w in [self.root]:
            w.configure(bg=bg)
        # all children
        for widget in self.root.winfo_children():
            try: widget.configure(bg=bg, fg=fg)
            except: pass
        # entries
        for e in [self.experimenter_entry, self.experiment_entry, self.json_entry, self.spreadsheet_entry]:
            e.configure(bg=ebg, fg=efg, insertbackground=efg)
        # log text
        self.log_text.configure(bg=tbg, fg=tfg, insertbackground=fg)
        # ports
        for port,ws in self.port_widgets.items():
            frame=ws['frame']; frame.configure(bg=bg)
            ws['status_label'].configure(bg=bg)
            ws['text_widget'].configure(bg=tbg, fg=tfg, insertbackground=fg)
            ws['indicator_canvas'].configure(bg=bg)
        # indicator
        self.canvas.configure(bg=cbg)

    def on_closing(self):
        if not self.data_saved:
            self.stop_identification_threads()
            self.stop_event.set()
            for t in list(self.identification_threads.values()): t.join()
            for t in list(self.port_threads.values()): t.join()
            self.save_all_data()
        self.root.destroy()

if __name__ == "__main__":
    splash = tk.Tk()
    SplashScreen(splash, duration=3000)
    splash.after(3000, splash.destroy)
    splash.mainloop()

    root = tk.Tk()
    app  = FED3MonitorApp(root)
    root.mainloop()
